In [57]:
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [261]:
data = np.load('../../data/othello_train_data.npz' ,allow_pickle=True)
X, y = data['X'], data['y']
X = np.where(X == 3, 0, X)



In [262]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train.reshape(-1, 64)).reshape(-1, 8, 8)
X_val = scaler.transform(X_val.reshape(-1, 64)).reshape(-1, 8, 8)

In [263]:
print("Training data shape:", X_train.shape, y_train.shape)
print("-" *10)

print(X_train[50])
print(f'This game winner was {y_train[50]}')

Training data shape: (47104, 8, 8) (47104,)
----------
[[ 1.  1. -1.  0.  1.  0. -1.  1.]
 [-1.  1.  0.  1.  1.  1.  0.  1.]
 [-1. -1.  1.  0.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  0.  0.  1.]
 [-1. -1.  1.  1.  1.  1.  0.  1.]
 [-1. -1.  1.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  1. -1.  1.]
 [-1.  0. -1.  0. -1.  1. -1. -1.]]
This game winner was -1


In [264]:
network = keras.Sequential([
    layers.Input(shape=(8,8,1)),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),

    layers.Flatten(),
    layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.Dropout(0.3),


    layers.Dense(1, activation='tanh')
    ])
network.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss='mse',metrics=['r2_score'])

In [265]:
early_stop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = network.fit(X_train, y_train, epochs=200, batch_size=32,validation_split=0.2,callbacks=[early_stop])

Epoch 1/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.8609 - r2_score: 0.0291 - val_loss: 0.7255 - val_r2_score: 0.1699
Epoch 2/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.6982 - r2_score: 0.2169 - val_loss: 0.6385 - val_r2_score: 0.3091
Epoch 3/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.5980 - r2_score: 0.3698 - val_loss: 0.5760 - val_r2_score: 0.4111
Epoch 4/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.5241 - r2_score: 0.4800 - val_loss: 0.5183 - val_r2_score: 0.4970
Epoch 5/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.4865 - r2_score: 0.5408 - val_loss: 0.4861 - val_r2_score: 0.5463
Epoch 6/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.4526 - r2_score: 0.5899 - val_loss: 0.4839 - val_r2_score: 0.5568
Epoch 7/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.4354 - r2_score: 0.6162 - val_loss: 0.4739 - val_r2_score: 0.5741
Epoch 8/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.4227 - r2_score: 0

In [266]:
network.evaluate(X_val, y_val)

368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3994 - r2_score: 0.6778


[0.40684130787849426, 0.6696897745132446]

In [267]:
#Uncomment below to save the  model
network.save("../../models/model_v1.keras")